In [ ]:
import mysql.connector
import pandas as pd
import datetime
from pandas.io import sql
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb as db

import numpy as np

In [ ]:
#### GNC ####
CAMPAIGN_ID = 23843180432940316
date_12 = datetime.date(2018,11,12)
date_13 = datetime.date(2018,11,13)
minute = datetime.time(11,59,20).minute

In [ ]:
def connectDB(db_name):
    mydb = mysql.connector.connect(
        host="localhost", 
        user="root",
        passwd="adgeek1234",
        database=db_name
    )
    return mydb
mydb = connectDB("ad_activity")
df_camp = pd.read_sql( "SELECT * FROM by_campaign where campaign_id = %s" %(CAMPAIGN_ID), con=mydb)

df_gnc = df_camp[(df_camp.request_time.dt.date == date_12) | (df_camp.request_time.dt.date == date_13)]
df_gnc = df_gnc[(df_gnc.request_time.dt.minute == minute)]

In [ ]:
#### 蜂主義 ####
CAMPAIGN_ID = 23843052139470246
date_12 = datetime.date(2018,11,12)
date_13 = datetime.date(2018,11,13)
minute = datetime.time(11,59,20).minute

In [ ]:
def connectDB(db_name):
    mydb = mysql.connector.connect(
        host="localhost", 
        user="root",
        passwd="adgeek1234",
        database=db_name
    )
    return mydb
mydb = connectDB("ad_activity")
df_camp = pd.read_sql( "SELECT * FROM by_campaign where campaign_id = %s" %(CAMPAIGN_ID), con=mydb)

df_feng = df_camp[(df_camp.request_time.dt.date == date_12) | (df_camp.request_time.dt.date == date_13)]
df_feng = df_feng[(df_feng.request_time.dt.minute == minute)]

In [ ]:
#### 善住 #####
CAMPAIGN_ID = 23843085681850632
date_12 = datetime.date(2018,11,12)
date_13 = datetime.date(2018,11,13)
minute = datetime.time(11,59,20).minute

In [ ]:
def connectDB(db_name):
    mydb = mysql.connector.connect(
        host="localhost", 
        user="root",
        passwd="adgeek1234",
        database=db_name
    )
    return mydb
mydb = connectDB("ad_activity")
df_camp = pd.read_sql( "SELECT * FROM by_campaign where campaign_id = %s" %(CAMPAIGN_ID), con=mydb)

df_shan = df_camp[(df_camp.request_time.dt.date == date_12) | (df_camp.request_time.dt.date == date_13)]
df_shan = df_shan[(df_shan.request_time.dt.minute == minute)]

In [ ]:
def connectDB(db_name):
    mydb = mysql.connector.connect(
        host="localhost", 
        user="root",
        passwd="adgeek1234",
        database=db_name
    )
    return mydb
mydb = connectDB("ad_activity")
df_camp = pd.read_sql( "SELECT * FROM by_campaign where campaign_id = %s" %(CAMPAIGN_ID), con=mydb)

In [ ]:
frames = [df_gnc, df_shan, df_feng]

In [ ]:
result = pd.concat(frames)

In [ ]:
len(result)

In [ ]:
def make_train(df):
    df['ratio'] = df['clicks'] / df['impressions']
    df.sort_values(by=['ad_id','request_time']).reset_index(drop=True)
    adid_list=df['ad_id'].unique()
    ad_list = []

    df_concat=pd.DataFrame(columns=df.columns)
    
    for ad in adid_list:
        df_tmp = df[df.ad_id==ad]
        if len(df_tmp) > 3:
            for i in range( 1,len( df_tmp ) ):
                df_tmp['clicks'].iloc[i-1] = df_tmp['clicks'].iloc[i] - df_tmp['clicks'].iloc[i-1]
                df_tmp['impressions'].iloc[i-1] = df_tmp['impressions'].iloc[i] - df_tmp['impressions'].iloc[i-1]
                df_tmp['reach'].iloc[i-1] = df_tmp['reach'].iloc[i] - df_tmp['reach'].iloc[i-1]
            df_concat=pd.concat([df_concat,df_tmp], ignore_index=True)

    for i in range(len(df['clicks'])):
        if df_concat['clicks'].iloc[i]==0:
            df_concat['clicks'].iloc[i]=df_concat['impressions'].iloc[i]*0.05
    
    for i in range(len(df['clicks'])):
        if df_concat['cpc'].iloc[i]==0:
            df_concat['cpc'].iloc[i]=np.mean(df['cpc'])
        
    df_drop = df_concat.drop(['age_max','age_min','flexible_spec','geo_locations', 'ratio', 'total_clicks','spend_cap', 'objective', 'start_time', 'stop_time', 'adset_id','campaign_id','avgspeed', 'speed'],axis=1)
    df_drop = df_drop.sort_values(by=['ad_id','request_time']).reset_index(drop=True)
    
    df_drop['request_time'] = pd.to_datetime(df_drop['request_time'], infer_datetime_format=True)
    hour = pd.get_dummies(df_drop['request_time'].dt.hour)
    train = pd.concat( [df_drop, hour] ,axis=1)
    train = train.reset_index(drop=True)
    
    return train

In [ ]:
TRAIN = make_train(result)
TRAIN.head(3)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import calendar
import random

In [ ]:
adid_list = TRAIN['ad_id'].unique()

In [ ]:
train_x=pd.DataFrame(columns=TRAIN.columns)
train_X=pd.DataFrame(columns=TRAIN.columns)
train_Xs=pd.DataFrame()
dfs=[]
for i in range(len(adid_list)):
    print(adid_list[i])
    train_x = TRAIN[(TRAIN.ad_id == adid_list[i]) ]
#     print(train_x.request_time.dt.hour)
    train_X = pd.concat( [ train_x.shift(1), train_x.shift(2), train_x.shift(3) ], axis=1, sort=False )

    train_X = train_X.dropna()
#     print(train_x['request_time'])
    train_X = train_X.drop(['request_time'], axis=1)

    train_Xs = pd.concat( [ train_Xs, train_X ],axis=0,sort=False)
    train_Xs = train_Xs.reset_index(drop=True)
    print('-----------------------',len(train_Xs))

In [ ]:
len(train_Xs)

In [ ]:
train_y=[]
train_Y=pd.DataFrame(columns=TRAIN.columns)
for i in range(len(adid_list)):
    print(adid_list[i])
    train_y = TRAIN[(TRAIN.ad_id == adid_list[i]) ]
#     print(train_y.request_time.dt.hour)
#     train_y = train_y[(TRAIN.ad_id == adid_list[i]) & (TRAIN.request_time.dt.hour != 1)]
#     train_y = train_y[(TRAIN.ad_id == adid_list[i]) & (TRAIN.request_time.dt.hour != 2)]
#     print(train.request_time.dt.hour)
#     train_y.drop(train_y.tail(3).index,inplace=True)
    train_y.drop(train_y.head(3).index,inplace=True)
    train_Y = pd.concat( [ train_Y, train_y ], ignore_index=True,sort=False )
    print('-----------------------',len(train_Y))
    train_Y = train_Y.drop(['request_time'], axis=1)
#     train_Y = train_Y.dropna()
train_Y = train_Y.reset_index(drop=True)
train_Y.head()

In [ ]:
train_Xs['ad_id'].iloc[0,0]

In [ ]:
train_Xs = train_Xs.drop(['ad_id'], axis=1)
train_Y = train_Y.drop(['ad_id'], axis=1)

In [ ]:
test_X = train_Xs.iloc[:21,:]
train_X = train_Xs.iloc[21:,:]

In [ ]:
test_Y = train_Y.iloc[:21,:]
train_Y = train_Y.iloc[21:,:]

In [ ]:
train_Y.columns

In [ ]:
def scaleData(train_X , train_Y):
    from sklearn.preprocessing import StandardScaler
    train_X_arr = train_X.values
    train_Y_arr = train_Y[['cpc','clicks']].values
#     train_Y_arr = train_Y[['clicks','cpc']].values
    #train_Y_arr = train_Y_arr.reshape(-1,1)

    scalerX = StandardScaler().fit(train_X_arr)
    scaled_X = scalerX.transform(train_X_arr)
    scalerY = StandardScaler().fit(train_Y_arr)
    scaled_Y = scalerY.transform(train_Y_arr)
    return scalerX, scaled_X, scalerY, scaled_Y

In [ ]:
scalerX, scaled_X, scalerY, scaled_Y = scaleData(train_X , train_Y[['cpc','clicks']] )

In [ ]:
from sklearn.externals import joblib
scalerX_filename = "IT_stateful_scalerX.pkl"
scalerY_filename = "IT_stateful_scalerY.pkl"
joblib.dump(scalerX, scalerX_filename) 
joblib.dump(scalerY, scalerY_filename) 

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras import optimizers, metrics
from keras.callbacks import EarlyStopping
from keras.models import load_model
from sklearn.externals import joblib 

BATCH_SIZE = 1
ITERATION = 10
EPOCH = 500
NEURONS = 128
MODEL_NAME = 'IT_test_cpc_' +  str(ITERATION) + '_' + str(EPOCH) + '_' + str(NEURONS) + '.h5'


In [ ]:
def create_model(train, target, batch_size, iteration, neurons, epoch):
    x, y = train, target
    x = train.reshape(train.shape[0], 1, train.shape[1])
    #x = train.values.reshape(train.shape[0], 1, train.shape[1])
    
    model = Sequential()
    model.add(LSTM( int(neurons) , batch_input_shape=(batch_size, x.shape[1], x.shape[2]), stateful = True, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(neurons, batch_input_shape=(batch_size, x.shape[1], x.shape[2]), stateful=True))
    model.add(Dropout(0.2))
    model.add(Dense(target.shape[1]))
    model.compile(loss='mse', optimizer='adam',metrics=[metrics.mse])
    
    es = EarlyStopping(monitor='val_loss', patience=10)
#     history = model.fit(x, y, epochs=EPOCH, batch_size=batch_size, verbose=1, shuffle=False, validation_split=0.2, callbacks=[es])
#     plt.plot(history.history['val_loss'], 'r')
    
    for i in range(iteration):
        print('====round', str(i) +  '/' + str(iteration))
        history = model.fit(x, y, epochs=EPOCH, batch_size=batch_size,
                  verbose=1, shuffle=False, validation_split=0.2, callbacks=[es])
        #plt.plot(history.history['val_loss'], 'r')
        model.reset_states()
    model.save(MODEL_NAME)
    return model, history

In [ ]:
%%time
model, history = create_model(scaled_X,scaled_Y, BATCH_SIZE, ITERATION, NEURONS, EPOCH)

In [ ]:
MODEL_NAME = '/storage/opt_project_test/optProjectTest/optProjectTest/models/IT_test_cpc_20_500_128(1).h5'
# MODEL_NAME = 'IT_stateful_alldata_20_500_128.h5'

In [ ]:
def evaluateModel(test_X,test_Y, scalerX, scalerY):
    #load model and scaler
    model = load_model(MODEL_NAME)
    
    
    
    
#     scalerX = joblib.load(scalerX_filename) 
#     scalerY = joblib.load(scalerY_filename)     
    
    test_X = scalerX.transform(test_X)

    test_X = test_X.reshape(test_X.shape[0],1,test_X.shape[1])
    pred_click, pred_cpc = [], []
    for i,arr in enumerate(test_X):
        ans = model.predict(test_X[i:i+1,:,:])
        ans = scalerY.inverse_transform(ans)
        print(ans)
#         pred_click.append(ans[0,0])
#         pred_cpc.append(ans[0,1])
        pred_cpc.append(ans[0])
    pred = pd.DataFrame({'pred_cpc':pred_cpc})
#     pred = pd.DataFrame({'pred_click':pred_click,'pred_cpc':pred_cpc})
    ans = pd.DataFrame(data=ans)
    frm = [pred,test_Y.reset_index(drop=True)]
    pred = pd.concat(frm,axis=1,sort=False)
    
#     pred['delta_click'] = pred['pred_click']-pred['clicks']
    pred['delta_cpc'] = pred['pred_cpc']-pred['cpc']
    return pred

In [ ]:
pred = evaluateModel(test_X, test_Y[['cpc','clicks']], scalerX, scalerY)
# print(pred.columns)
# pred[['pred_click', 'clicks', 'delta_click' , 'pred_cpc' , 'cpc', 'delta_cpc', 'impressions']]
# pred[['pred_cpc' , 'cpc', 'delta_cpc', 'impressions']]
pred.head(10)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(pred['cpc'])
plt.plot(pred['pred_cpc'])
# plt.plot(pred['delta_cpc'])

In [ ]:
plt.figure(figsize=(15,3))
plt.plot(pred['cpc'])
plt.plot(pred['pred_cpc'])
# plt.plot(pred['delta_cpc'])

In [ ]:
plt.figure(figsize=(7,3))
plt.plot(pred['cpc'])
plt.plot(pred['pred_cpc'])
# plt.plot(pred['delta_cpc'])

In [ ]:
plt.figure(figsize=(15,3))
plt.plot(pred['cpc'])
plt.plot(pred['pred_cpc'])
# plt.plot(pred['delta_cpc'])

In [ ]:
plt.figure(figsize=(7,3))
plt.plot(pred['cpc'])
plt.plot(pred['pred_cpc'])
# plt.plot(pred['delta_cpc'])

In [ ]:
plt.figure(figsize=(15,3))
plt.plot(pred['cpc'].tail(200))
plt.plot(pred['pred_cpc'].tail(200))
# plt.plot(pred['delta_cpc'])

In [ ]:
train_Y[['impressions', 'daily_budget']].tail(200)

In [1]:
def normalized_sigmoid_fkt(a, b, x):
    s= 1/(1+np.exp( b*(x-a) ))
    return s

In [ ]:
3 * 0.8* ( normalized_sigmoid_fkt(a, b, 9) - 0.5 ) +3

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
a = 1
b = 5
bid_range = 0.8
init_cpc = 500


x = np.linspace(0, 2)
# x_new = np.linspace(0, 1)

tanh = np.linspace(0, 1)
y = normalized_sigmoid_fkt(a, b, x)
# for x_index in range(len(x)):
#     if x[x_index] >= 0.5:    
#         x_new[x_index]= 1 - x[x_index]
#         tanh[x_index] = init_cpc * bid_range * ( normalized_sigmoid_fkt(a, b, x_new[x_index]) - 0.5 ) + init_cpc
#     else:
#         tanh[x_index]  = init_cpc * bid_range * ( normalized_sigmoid_fkt(a, b, x_new[x_index]) - 0.5 ) + init_cpc
# ratio = 
# tanh = init_cpc * ratio

# y1 = np.tanh(x)
# plt.plot(x,y,label="Tanh", color = "red")
plt.plot(x,tanh,label="Tanh", color = "blue")

In [17]:
bid_cap = 550
ratio = BID_RANGE * ( normalized_sigmoid_fkt(CENTER, WIDTH, 0) - 0.5 ) + 1
init_bid = bid_cap/ratio
init_bid

392.8673344798104

In [5]:
ratio

1.399963681705038

In [15]:
bid_amount = 550
from bid_operator import *
def reverse_bid_amount(bid_amount):
    init_bid = bid_amount / ( BID_RANGE * ( normalized_sigmoid_fkt(CENTER, WIDTH, 0) - 0.5 ) + 1 )
    return init_bid

In [10]:
( BID_RANGE * ( normalized_sigmoid_fkt(CENTER, WIDTH, 0) - 0.5 ) + 1)

1.399963681705038

In [16]:
reverse_bid_amount(bid_amount)

392.8673344798104

In [12]:
bid = reverse_bid_amount(bid_amount)

In [ ]:
import math
bid=math.ceil(bid)
bid=bid*1.1

In [18]:
r = bid_amount * BID_RANGE * ( normalized_sigmoid_fkt(CENTER, WIDTH, 0) - 0.5 )+ bid_amount

In [19]:
r

769.980024937771

In [ ]:
( normalized_sigmoid_fkt(a, b, 0.5) )
plt.plot(x,r,label="Tanh", color = "blue")